# Zillow Data via Quandl

In [3]:
import os
import sys
import quandl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# sys.path.append('C:Users/Rob/repos/labspt15-cityspire-g-ds')
# sys.path

# path = r'C:\Users\Rob\repos\\'
# path == r'V:\\'

In [5]:
# quandl.ApiConfig.api_key = 'rfzykh-NDsw72dmc4Ms4'

In [7]:
# data = quandl.get_table('MER/F1', compnumber="39102", paginate=True)

In [9]:
# quandl.ApiConfig.api_key = 'rfzykh-NDsw72dmc4Ms4'
# quandl.get_table('ZILLOW/DATA', indicator_id='ZSFH', region_id='99999')

## Data

In [11]:
data = pd.read_csv("V:\ZILLOW_DATA.csv")
data.head()

,indicator_id,region_id,date,value
0,ZATT,3101,1998-01-31,338849.0
1,ZATT,3101,1998-02-28,342993.0
2,ZATT,3101,1998-03-31,346763.0
3,ZATT,3101,1998-04-30,349356.0
4,ZATT,3101,1998-05-31,351981.0


In [13]:
len(data) #125.5 million lines!!

125528409

In [15]:
data['indicator_id'].unique()

array(['ZATT', 'ZSFH', 'ZALL', 'ZCON', 'ZABT', 'Z5BR', 'Z2BR', 'Z3BR',
       'Z1BR', 'Z4BR', 'ISAW', 'SSAW', 'IRAM', 'CRSM', 'CSSM', 'MSAM',
       'IRAW', 'MRAM', 'SSSW', 'LSSM', 'CSAW', 'ISSM', 'SASW', 'MRAW',
       'LRAM', 'ISSW', 'MSAW', 'LSAM', 'CRAM', 'ISAM', 'IRSM', 'SRSW',
       'LSAW', 'NRAW', 'NSAW', 'CRSW', 'SSAM', 'IRSW', 'NRAM', 'CRAW',
       'LRSM', 'SASM', 'SSSM', 'SAAM', 'LRAW', 'CSAM', 'SRAW', 'CSSW',
       'LRSW', 'SRSM', 'SAAW', 'LSSW', 'SRAM', 'NSAM', 'RSNA', 'RSSA'],
      dtype=object)

In [17]:
rental_data = data[data['indicator_id'].isin(['RSSA', 'RSNA'])]

In [19]:
print(len(rental_data))
print(rental_data.shape)
rental_data.head()

535891
(535891, 4)


,indicator_id,region_id,date,value
1586389,RSNA,102001,2020-06-30,1777.0
1586392,RSNA,394913,2020-06-30,2811.0
1586396,RSNA,753899,2020-06-30,2625.0
1586400,RSNA,394463,2020-06-30,1804.0
1586403,RSNA,394514,2020-06-30,1611.0


In [21]:
print('average rent: ', round(sum(rental_data['value']) / len(rental_data)))

average rent:  1686


## Indicators

In [23]:
indicators = pd.read_csv('V:\ZILLOW_INDICATORS.csv')
indicators.head()

,indicator_id,indicator,category
0,ZATT,ZHVI All Homes- Top Tier Time Series ($),Home values
1,ZSFH,ZHVI Single-Family Homes Time Series ($),Home values
2,ZALL,"ZHVI All Homes (SFR, Condo/Co-op) Time Series ($)",Home values
3,ZCON,ZHVI Condo/Co-op Time Series ($),Home values
4,ZABT,ZHVI All Homes- Bottom Tier Time Series ($),Home values


In [25]:
indicators['indicator_id'].unique()

array(['ZATT', 'ZSFH', 'ZALL', 'ZCON', 'ZABT', 'Z5BR', 'Z2BR', 'Z3BR',
       'Z1BR', 'LSAW', 'NRAW', 'NSAW', 'CRSW', 'SSAM', 'IRSW', 'NRAM',
       'CRAW', 'LRSM', 'SASM', 'SSSM', 'SAAM', 'LRAW', 'CSAM', 'SRAW',
       'CSSW', 'LRSW', 'SRSM', 'SAAW', 'LSSW', 'SRAM', 'NSAM', 'RSNA',
       'RSSA', 'Z4BR', 'ISAW', 'SSAW', 'IRAM', 'CRSM', 'CSSM', 'MSAM',
       'IRAW', 'MRAM', 'SSSW', 'LSSM', 'CSAW', 'ISSM', 'SASW', 'MRAW',
       'LRAM', 'ISSW', 'MSAW', 'LSAM', 'CRAM', 'ISAM', 'IRSM', 'SRSW'],
      dtype=object)

In [27]:
indicators

,indicator_id,indicator,category
0,ZATT,ZHVI All Homes- Top Tier Time Series ($),Home values
1,ZSFH,ZHVI Single-Family Homes Time Series ($),Home values
2,ZALL,"ZHVI All Homes (SFR, Condo/Co-op) Time Series ($)",Home values
3,ZCON,ZHVI Condo/Co-op Time Series ($),Home values
4,ZABT,ZHVI All Homes- Bottom Tier Time Series ($),Home values
5,Z5BR,ZHVI 5+ Bedroom Time Series ($),Home values
6,Z2BR,ZHVI 2-Bedroom Time Series ($),Home values
7,Z3BR,ZHVI 3-Bedroom Time Series ($),Home values
8,Z1BR,ZHVI 1-Bedroom Time Series ($),Home values
9,LSAW,"Median List Price (Smooth, All Homes, Weekly V...",Inventory and sales


In [29]:
indicators['category'].unique()

array(['Home values', 'Inventory and sales', 'Rentals'], dtype=object)

In [31]:
rentals = indicators[indicators['category'] == 'Rentals']
rentals.head()

,indicator_id,indicator,category
31,RSNA,ZORI (Smoothed): All Homes Plus Multifamily Ti...,Rentals
32,RSSA,"ZORI (Smoothed, Seasonally Adjusted): All Home...",Rentals


In [67]:
for item in rentals['indicator_id']:
    print(item)

RSNA
RSSA


## Regions

In [35]:
regions = pd.read_csv('V:\ZILLOW_REGIONS.csv')
regions = regions.sort_values(by=['region_id'])
regions.head(20)

,region_id,region_type,region
1150,3,state,Alaska; AK
453,4,state,Alabama; AL
515,6,state,Arkansas; AR
30,8,state,Arizona; AZ
10,9,state,California; CA
2555,10,state,Colorado; CO
349,11,state,Connecticut; CT
1243,12,state,District of Columbia; DC
1149,13,state,Delaware; DE
11,14,state,Florida; FL


In [37]:
len(regions)

78730

In [39]:
regions['region_type'].unique()

array(['state', 'county', 'city', 'neigh', 'zip', 'metro'], dtype=object)

In [41]:
len(regions['region_type'] == 'metro')

78730

In [43]:
metro = regions[regions['region_type'] == 'metro']
metro[:50]

,region_id,region_type,region
5176,102001,metro,United States
13277,394297,metro,"Aberdeen, SD; SD"
13668,394298,metro,"Aberdeen, WA; WA"
1213,394299,metro,"Abilene, TX; TX"
4449,394300,metro,"Ada, OK; OK"
3469,394302,metro,"Adrian, MI; MI"
1237,394304,metro,"Akron, OH"
5831,394305,metro,"Alamogordo, NM; NM"
2423,394306,metro,"Albany, GA; GA"
2899,394307,metro,"Albany, OR; OR"


# Data.World Population Data Set

In [45]:
population = pd.read_excel('https://query.data.world/s/gewikohoj4s2gqv4bowlzpdxeu4jzl', usecols=['State', 'City', 'Population'])
population.head()

,State,City,Population
0,Alabama,Abernant,7599
1,Alabama,Alabaster,31545
2,Alabama,Albertville,21458
3,Alabama,Alexander City,14849
4,Alabama,Andalusia,9081


In [47]:
len(population)

6889

In [49]:
population['City']

0             Abernant
1            Alabaster
2          Albertville
3       Alexander City
4            Andalusia
             ...      
6884            Lander
6885        Torrington
6886           Douglas
6887            Powell
6888    Bridger Valley
Name: City, Length: 6889, dtype: object

In [51]:
cities = population[population['Population'] > 80000]
len(cities)

500

In [53]:
len(cities['State'].unique())

47

In [55]:
cities['State'].unique()

array(['Alabama', 'Alaska', 'Arkansas', 'Arizona', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'DC', 'Florida', 'Georgia',
       'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
       'Kentucky', 'Louisiana', 'Maryland', 'Massachusetts', 'Michigan',
       'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska',
       'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'Utah', 'Virginia', 'Washington',
       'Wisconsin'], dtype=object)

In [57]:
states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 
'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 
'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 
'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 
'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire','New Jersey', 
'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 
'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 
'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 
'West Virginia', 'Wisconsin', 'Wyoming']

In [59]:
missing = []

for state in states:
    if state not in list(cities['State'].unique()):
        missing.append(state)

print(missing)

['Maine', 'Vermont', 'West Virginia', 'Wyoming']


# Aggregating Data

In [61]:
# Checking if data has all the same sections
sorted(data['indicator_id'].unique()) == sorted(indicators['indicator_id'].unique())

True

In [96]:
rental_data = rental_data.sort_values(by=['region_id'])
rental_data = rental_data[rental_data['indicator_id'].isin(['RSSA', 'RSNA'])]
print(rental_data.shape)
rental_data.head()

(535891, 4)


,indicator_id,region_id,date,value
7851413,RSNA,58362,2020-09-30,1321.0
7854977,RSSA,58362,2020-10-31,1363.0
7851412,RSNA,58362,2020-10-31,1382.0
8753680,RSSA,58362,2020-05-31,1243.0
8753679,RSSA,58362,2020-06-30,1251.0


,indicator_id,region_id,date,value
7851413,RSNA,58362,2020-09-30,1321.0
8753679,RSSA,58362,2020-06-30,1251.0
8747088,RSNA,58362,2020-06-30,1220.0
8747082,RSNA,58362,2020-07-31,1351.0
8747089,RSNA,58362,2020-05-31,1243.0
...,...,...,...,...
51008783,RSSA,753924,2019-10-31,2105.0
1610766,RSSA,753924,2020-04-30,2093.0
51008768,RSSA,753924,2018-07-31,2057.0
51008757,RSSA,753924,2017-08-31,2029.0


In [98]:
rent_per_region = pd.merge(rental_data, regions, on='region_id')
rent_per_region.head()

,indicator_id,region_id,date,value,region_type,region
0,RSNA,58362,2020-09-30,1321.0,zip,01432; MA; Boston-Cambridge-Newton; Middlesex ...
1,RSSA,58362,2020-10-31,1363.0,zip,01432; MA; Boston-Cambridge-Newton; Middlesex ...
2,RSNA,58362,2020-10-31,1382.0,zip,01432; MA; Boston-Cambridge-Newton; Middlesex ...
3,RSSA,58362,2020-05-31,1243.0,zip,01432; MA; Boston-Cambridge-Newton; Middlesex ...
4,RSSA,58362,2020-06-30,1251.0,zip,01432; MA; Boston-Cambridge-Newton; Middlesex ...


In [100]:
len(rent_per_region['region'].unique())

4031

In [113]:
rent_per_region[[
    'zip', 'state', 'city', 'county', 'neighborhood'
    ]] = rent_per_region['region'].str.split(';', expand=True)

In [120]:
test = rent_per_region[::-1]
test = test[:100]
test.head(20)

,indicator_id,region_id,date,value,region_type,region,zip,state,city,county,neighborhood
535890,RSSA,753924,2017-04-30,2023.0,metro,"Urban Honolulu, HI","Urban Honolulu, HI",None,None,None,None
535889,RSNA,753924,2018-08-31,2129.0,metro,"Urban Honolulu, HI","Urban Honolulu, HI",None,None,None,None
535888,RSNA,753924,2018-12-31,2110.0,metro,"Urban Honolulu, HI","Urban Honolulu, HI",None,None,None,None
535887,RSNA,753924,2017-08-31,2082.0,metro,"Urban Honolulu, HI","Urban Honolulu, HI",None,None,None,None
535886,RSNA,753924,2020-03-31,2172.0,metro,"Urban Honolulu, HI","Urban Honolulu, HI",None,None,None,None
535885,RSNA,753924,2020-04-30,2152.0,metro,"Urban Honolulu, HI","Urban Honolulu, HI",None,None,None,None
535884,RSNA,753924,2020-06-30,2101.0,metro,"Urban Honolulu, HI","Urban Honolulu, HI",None,None,None,None
535883,RSNA,753924,2015-06-30,2006.0,metro,"Urban Honolulu, HI","Urban Honolulu, HI",None,None,None,None
535882,RSNA,753924,2017-06-30,2080.0,metro,"Urban Honolulu, HI","Urban Honolulu, HI",None,None,None,None
535881,RSNA,753924,2017-05-31,2067.0,metro,"Urban Honolulu, HI","Urban Honolulu, HI",None,None,None,None


In [121]:
test[['city', 'state']] = test['region'].str.split(',', expand=True)
test.head()

,indicator_id,region_id,date,value,region_type,region,zip,state,city,county,neighborhood
535890,RSSA,753924,2017-04-30,2023.0,metro,"Urban Honolulu, HI","Urban Honolulu, HI",HI,Urban Honolulu,None,None
535889,RSNA,753924,2018-08-31,2129.0,metro,"Urban Honolulu, HI","Urban Honolulu, HI",HI,Urban Honolulu,None,None
535888,RSNA,753924,2018-12-31,2110.0,metro,"Urban Honolulu, HI","Urban Honolulu, HI",HI,Urban Honolulu,None,None
535887,RSNA,753924,2017-08-31,2082.0,metro,"Urban Honolulu, HI","Urban Honolulu, HI",HI,Urban Honolulu,None,None
535886,RSNA,753924,2020-03-31,2172.0,metro,"Urban Honolulu, HI","Urban Honolulu, HI",HI,Urban Honolulu,None,None


In [122]:
rent_per_region[['city', 'state']] = rent_per_region['region'].str.split(',', expand=True)

,indicator_id,region_id,date,value,region_type,region,zip,state,city,county,neighborhood
535886,RSNA,753924,2020-03-31,2172.0,metro,"Urban Honolulu, HI","Urban Honolulu, HI",HI,Urban Honolulu,None,None
535887,RSNA,753924,2017-08-31,2082.0,metro,"Urban Honolulu, HI","Urban Honolulu, HI",HI,Urban Honolulu,None,None
535888,RSNA,753924,2018-12-31,2110.0,metro,"Urban Honolulu, HI","Urban Honolulu, HI",HI,Urban Honolulu,None,None
535889,RSNA,753924,2018-08-31,2129.0,metro,"Urban Honolulu, HI","Urban Honolulu, HI",HI,Urban Honolulu,None,None
535890,RSSA,753924,2017-04-30,2023.0,metro,"Urban Honolulu, HI","Urban Honolulu, HI",HI,Urban Honolulu,None,None


In [131]:
len(rent_per_region['state'].unique())

42

In [132]:
rent_per_region.isnull().sum()

indicator_id         0
region_id            0
date                 0
value                0
region_type          0
region               0
zip                  0
state           517923
city                 0
county          380909
neighborhood    393657
dtype: int64

In [140]:
count = 0
for i in rent_per_region['zip']:
    if i.isdigit():
        count += 1

print(count)

01432
01432
01432
01432
01432
01432
01432
01432
01432
01432
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01545
01603
01603
01603
01603
01603
01603
01603
01603
01603
01603
0160